In [4]:
from keras.layers import Input, Dense, Activation, Flatten, Dropout, Reshape
from keras.layers import Lambda, LSTM
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Model
import keras.backend as K
import keras

from training_utils import root_mean_squared_error, save_model, DataGenerator


# Global Vairable

In [5]:
IMAGE_SHAPE = (52, 69, 1)
LSTM_NUM_HIDDEN_STATE = 3 * 192  # for LSTM
PLANNING_HORIZON = 25

___

# Model Definition

Here, Encoder & Decoder is define so that they can  be shared

**--------------------------------------------**

## Encoder

In [7]:
def encoder_seed_style(img_shape):
    """
    Definition of a CNN according to seed paper to encode 3 images
    
    Input:
        img_shape (tuple):
    """
    
    def _block(X, nb_filters, kernel_size, strides):
        """
        Define a block in the encoder 
        
        Input:
            X (Tensor)
            nb_filters (int): nb of filters of Conv2D layers (2 Con2D layers have the same nb. of filters)
            kernel_size (tuple): size of a filter of a Conv2D layer
            strides (tuple): strides of a Conv2D layer
        """
        X = Conv2D(nb_filters, kernel_size, strides=strides, padding='valid', activation='relu')(X)
        X = Conv2D(nb_filters, kernel_size, strides=strides, padding='valid', activation='relu')(X)
        X = BatchNormalization()(X) 
        X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(X)
        return X
    
    # Input layer
    X_input = Input(shape=img_shape)
    
    # Normalize layer
    X = Lambda(lambda x: x / 127.5 - 1.0)(X_input)
    
    # First block
    X = _block(X, 24, kernel_size=(3, 3), strides=(1, 1))
    
    # Second block
    X = _block(X, 36, kernel_size=(3, 3), strides=(1, 1))
    
    # Third block
    X = _block(X, 48, kernel_size=(3, 3), strides=(1, 1))
    
    # Fourth block
    X = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(X)
    X = Conv2D(64, (3, 3), strides=(1, 1), padding='valid', activation='relu')(X)
    
    y = Flatten()(X)
    
    # Define model
    model = Model(inputs=[X_input], outputs=[y])
    
    return model
    

In [8]:
# Create encoder
encoder = encoder_seed_style(IMAGE_SHAPE)
encoder.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 52, 69, 1)         0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 52, 69, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 67, 24)        240       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 65, 24)        5208      
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 65, 24)        96        
_________________________________________________________________
max_pooling2d_1 (M

**--------------------------------------------**

## Decoder

In [9]:
LSTM_cell = LSTM(LSTM_NUM_HIDDEN_STATE, return_state=True)

reshapor = Reshape((1, -1))


def _regressor(input_shape):
    """
    Define a stack of Dense layers to regress LSTM hidden state to actual steering angle 
    """
    X_input = Input(shape=input_shape)
    
    X = Dense(10, activation='relu')(X_input)
    
    y = Dense(1, activation=None)(X)
    
    model = Model(inputs=[X_input], outputs=[y])
    
    return model

regressor = _regressor(input_shape=(LSTM_NUM_HIDDEN_STATE,))

In [10]:
regressor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5770      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 5,781
Trainable params: 5,781
Non-trainable params: 0
_________________________________________________________________


**--------------------------------------------**
## Put encoder & decoder together 

In [11]:
def hybrid_LSTM_training(img_shape):
    """
    Define full hybrid model - training architect
    """
    # Input layer
    X_in_0 = Input(shape=img_shape)
    X_in_1 = Input(shape=img_shape)
    X_in_2 = Input(shape=img_shape)
    
    # Encode
    X_0 = encoder(X_in_0)
    X_1 = encoder(X_in_1)
    X_2 = encoder(X_in_2)
    
    # initial input to decoder ( = LSTM following by regressor)
    a_0 = keras.layers.concatenate([X_0, X_1, X_2])
    
    # initialize input & cell state
    x_0 = Input(shape=(1, 1))  
    c_0 = Input(shape=(LSTM_NUM_HIDDEN_STATE,))
    
    x = x_0
    a = a_0
    c = c_0
    _outputs = []
    
    # Decode
    for t in range(PLANNING_HORIZON):
        # perform 1 step of LSTM cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        
        # apply regressor to the hidden state of LSTM_cell
        out = regressor(a)
        
        # append out to outputs
        _outputs.append(out)
        
        # update input for next step of LSTM_cell
        x = reshapor(out)
    
    outputs = keras.layers.concatenate(_outputs)
    
    # define model
    model = Model(inputs=[X_in_0, X_in_1, X_in_2, x_0, c_0], outputs=outputs)
    
    return model

In [12]:
hybrid_model = hybrid_LSTM_training(IMAGE_SHAPE)

In [13]:
hybrid_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 52, 69, 1)    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 52, 69, 1)    0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 52, 69, 1)    0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 192)          126416      input_3[0][0]                    
                                                                 input_4[0][0]                    
          

In [19]:
# choose optimizer & compile model
optim = keras.optimizers.Adam(decay=1e-4)

hybrid_model.compile(loss=root_mean_squared_error, 
                     optimizer=optim, 
                     metrics=['mae'])

***
# Training

Baseline:
* RMSE: 0.09
* MAE: 0.03
* EVA: 0.70

In [6]:
params = {'img_shape': IMAGE_SHAPE, 
          'batch_size': 1, 
          'shuffle': True,
          'additional_input_for_LSTM': True,
         'LSTM_nb_hidden_states': LSTM_NUM_HIDDEN_STATE}

training_generator = DataGenerator('./data/ext_hybrid_training.csv', **params)
validation_generator = DataGenerator('./data/ext_hybrid_validation.csv', **params)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='mean_absolute_error', restore_best_weights=False)
hybrid_model.fit_generator(training_generator, 
                           epochs=1, 
                           verbose=1, 
                           validation_data=validation_generator, 
                           callbacks=[early_stopping])

Epoch 1/1
 7711/14486 [==============>...............] - ETA: 10:26 - loss: 0.0748 - mean_absolute_error: 0.0697

In [18]:
# save model
# model name convention: rw_EncoderArchitect_DecoderArchitect_Date_Time_OrderOfEpoch
save_model(hybrid_model, "rw_SeedPaper_LSTM_MAY16_18h??")

/home/user/venv/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'concatenate_1/concat:0' shape=(?, ?) dtype=float32>, <tf.Tensor 'input_7:0' shape=(?, 576) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/user/venv/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/TensorArrayReadV3:0' shape=(?, 576) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_4:0' shape=(?, 576) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/user/venv/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword argum